In [1]:
import RNA
import numpy as np

import subprocess
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import SVG, display
from collections import Counter
from collections import defaultdict

import difflib
import sys
import os
import random
import string
import time

# from helper import print_moves

import pandas as pd

sys.path.append('../')
from pretty_print_path import print_moves
import findpath_librna
import findpath


In [2]:
%load_ext autoreload
%autoreload 2
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True) # no scientific notation

# import feature_generation
from features import ij_distance, new_move_dist, plt_moves, config_distance, balance_in_all_things, return_shift_moves
from process_features import fp_call, find_moves, process

In [44]:
sequence = 'AGGUGUUAUAAAAGUGCAGAGCCAAGUCACCGCCUUGCCGUCGCUUCCGCUAUGCCUCAAAAGCGGAAUAUCGAGCGAGCGAUUGUGAUUCGAAUGGUCG'
s1       = '.(((................))).((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..))))))..........'
s2       = '...(((.........)))..((((((((((.(((((((((....(((((((.(......).)))))))...)).))))).).).))))))....))))..'

search_width = 60
# search_width = 500

s = s1

sw = 2
fp = findpath.findpath_single(sequence, s1, s2, search_width_multiplier=sw, mp=True)
result = fp.get_en()/100.0
path = fp.get_path()


add_moves        = set()
delete_moves     = set()
available_add    = set()
available_delete = set()

en_contrib = []

for e, tup in enumerate(path):
    i = tup[0]
    j = tup[1]
    if i<0:
        delete_moves.add((i,j))
    if i>0:
        add_moves.add((i,j))

## we are evaluating 1 node = 1 structure
## means we are evaluating ALL moves (i, j) and see if we can find any patterns

##np.mean(en_contrib) ## mean of energies
##np.std(en_contrib) ## diversity of energies


## verhaeltnis delete add moves bezogen auf accessible moves
## mean of energy
## how many add / delete moves are currently available?
## if len(available_delete)/len(delete_moves) is high, the probability
## of making a wrong decision is probably quite high. 

## -------
## true values for classifier:
## - percentage of moves which lead to the optimal result?
## - sort by energy, where is the first contribution which leads to the optimal result? 
## - look at top energy contributions: how long do we have to postpone them to get the optimal result? 


fc = RNA.fold_compound(sequence)
pt = list(RNA.ptable(s))
pt1 = list(RNA.ptable(s1))
pt2 = list(RNA.ptable(s2))

for pos, (i,j) in enumerate(find_moves(pt, pt2)):  
    if (i,j) in add_moves:
        available_add.add((i,j))
    if (i,j) in delete_moves:
        available_delete.add((i,j))
    
    # map energies somehow to [0,1] such that the network will understand.  
    en = np.interp(fc.eval_move_pt(pt, i, j)/100, [-10,10], [0,1]) 
    en_contrib.append(en)

available_add, available_delete, len(available_add)/len(add_moves), len(available_delete)/len(delete_moves)

np.std(en_contrib)

0.07859048825239605

0.10750000000000003

In [23]:
# [   0,    0 ] .(((................))).((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..))))))..........  -23.10 0.125
# [  -4,  -21 ] .((..................)).((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..))))))..........  -20.70 0.1875
# [  -3,  -22 ] .(....................).((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..))))))..........  -17.70 0.0
# [  -2,  -23 ] ........................((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..))))))..........  -21.90 0.05882
# [  23,   96 ] ......................(.((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..)))))).....)....  -18.10 0.0
# [  22,   97 ] .....................((.((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..)))))).....))...  -21.00 0.0
# [  24,   95 ] .....................(((((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..))))))....)))...  -23.60 0.0
# [  21,   98 ] ....................((((((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..))))))....))))..  -25.20 0.07692
# [ -35,  -82 ] ....................((((((((((.....((((((((.(((((((.(......).)))))))...))).)).)))...))))))....))))..  -23.60 0.08333
# [ -36,  -81 ] ....................((((((((((......(((((((.(((((((.(......).)))))))...))).)).))....))))))....))))..  -22.70 0.1667
# [ -37,  -80 ] ....................((((((((((.......((((((.(((((((.(......).)))))))...))).)).).....))))))....))))..  -19.10 0.0
# [ -38,  -79 ] ....................((((((((((........(((((.(((((((.(......).)))))))...))).)).......))))))....))))..  -20.00 0.1667
# [ -39,  -77 ] ....................((((((((((.........((((.(((((((.(......).)))))))...))).)........))))))....))))..  -17.40 0.0
# [ -40,  -76 ] ....................((((((((((..........(((.(((((((.(......).)))))))...)))..........))))))....))))..  -19.70 0.0
# [  32,   83 ] ....................((((((((((.(........(((.(((((((.(......).)))))))...)))........).))))))....))))..  -18.10 0.0
# [  33,   81 ] ....................((((((((((.((.......(((.(((((((.(......).)))))))...)))......).).))))))....))))..  -17.10 0.0
# [  34,   79 ] ....................((((((((((.(((......(((.(((((((.(......).)))))))...)))....).).).))))))....))))..  -16.30 0.0
# [  35,   78 ] ....................((((((((((.((((.....(((.(((((((.(......).)))))))...)))...)).).).))))))....))))..  -17.90 0.0
# [  36,   77 ] ....................((((((((((.(((((....(((.(((((((.(......).)))))))...)))..))).).).))))))....))))..  -19.50 0.0
# [  37,   76 ] ....................((((((((((.((((((...(((.(((((((.(......).)))))))...))).)))).).).))))))....))))..  -21.50 0.0
# [  38,   75 ] ....................((((((((((.(((((((..(((.(((((((.(......).)))))))...)))))))).).).))))))....))))..  -24.60 0.0
# [ -41,  -74 ] ....................((((((((((.(((((((...((.(((((((.(......).)))))))...)).))))).).).))))))....))))..  -23.20 0.0
# [ -42,  -73 ] ....................((((((((((.(((((((....(.(((((((.(......).)))))))...)..))))).).).))))))....))))..  -19.90 0.0
# [ -43,  -72 ] ....................((((((((((.(((((((......(((((((.(......).)))))))......))))).).).))))))....))))..  -22.70 0.0
# [  39,   73 ] ....................((((((((((.((((((((.....(((((((.(......).)))))))....).))))).).).))))))....))))..  -21.90 0.0
# [  40,   72 ] ....................((((((((((.(((((((((....(((((((.(......).)))))))...)).))))).).).))))))....))))..  -25.20 0.0
# [   5,   17 ] ....(...........)...((((((((((.(((((((((....(((((((.(......).)))))))...)).))))).).).))))))....))))..  -21.20 0.0
# [   6,   16 ] ....((.........))...((((((((((.(((((((((....(((((((.(......).)))))))...)).))))).).).))))))....))))..  -23.70 0.0

In [43]:
# which nodes do we permutate first? in principle, this is a simlar concept as the findpath search width, where we just
# use the sw. to perform a look-ahead

## IDEA
# check how many moves out of all choices are net-negative. 


def adjust_pt(pt, i, j):
    pt_adj = pt.copy()
    if i<0:
        pt_adj[-i] = 0
        pt_adj[-j] = 0
    else:
        pt_adj[i] = j
        pt_adj[j] = i
    return pt_adj


fc = RNA.fold_compound(sequence)
pt = list(RNA.ptable(s1))
pt1 = list(RNA.ptable(s1))
pt2 = list(RNA.ptable(s2))

for i,j in find_moves(pt1, pt2):
    
    pt = adjust_pt(pt1, i, j)
    s = RNA.db_from_ptable(pt)


    search_width = 1
    sE, path = findpath_librna.pathfinder(sequence, s, s2, search_width=search_width, return_paths=True)
    
    en = fc.eval_move_pt(pt1, i, j)

    # print_moves(sequence, s, s2, path)
    print (s, i, j, en, sE)

    # break




..((................))..((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..)))))).......... -2 -23 380 -7.1
.(.(................).).((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..)))))).......... -3 -22 700 -7.1
.((..................)).((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..)))))).......... -4 -21 240 -7.1
.((((...........)...))).((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..)))))).......... 5 17 310 -6.5
.(((.(.........)....))).((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..)))))).......... 6 16 460 -6.5
.(((................)))(((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..))))))....)..... 24 95 450 -5.5
.(((................))).((((((.(..(((((((((.(((((((.(......).)))))))...))).)).))))).)))))).......... 32 83 110 -7.1
.(((................))).((((((.....((((((((.(((((((.(......).)))))))...))).)).)))...)))))).......... -35 -82 160 -7.1
.(((................))).((((((....(.(((((((.(((((((.(......).))))))).